In [76]:
import cv2
import numpy as np

In [77]:
def min_filter_mean(src, r=7):
    """
    最小值滤波，
    :param src: 源图像
    :param r: r是滤波器半径
    :return:
    """
    return cv2.erode(src, np.ones((2 * r + 1, 2 * r + 1)))


def guided_filter(image, p, r, eps):
    m_i = cv2.boxFilter(image, -1, (r, r))
    m_p = cv2.boxFilter(p, -1, (r, r))
    m_ip = cv2.boxFilter(image * p, -1, (r, r))
    cov_ip = m_ip - m_i * m_p

    m_ii = cv2.boxFilter(image * image, -1, (r, r))
    var_i = m_ii - m_i * m_i

    a = cov_ip / (var_i + eps)
    b = m_p - a * m_i

    m_a = cv2.boxFilter(a, -1, (r, r))
    m_b = cv2.boxFilter(b, -1, (r, r))
    return m_a * image + m_b

# 输入rgb图像，值范围[0,1]
def fog(m, r, eps, w, max_v1):
    """
    计算大气遮罩图像V1和光照值A, V1 = 1-t/A'''
    """
    image_dark = np.min(m, 2)
    # 得到暗通道图像
    dark_channel = min_filter_mean(image_dark, 5)
    dark_channel_ori = dark_channel * 255
    cv2.imwrite('worker1/result/1_dark.png', dark_channel_ori)
    # 查看暗通道
    # cv2.imshow('Dark',dark_channel)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # 使用引导滤波优化
    image_dark = guided_filter(image_dark, dark_channel, r, eps)
    bins = 2000
    # 计算大气光照A
    ht = np.histogram(image_dark, bins)
    d = np.cumsum(ht[0]) / float(image_dark.size)
    for lmax in range(bins - 1, 0, -1):
        if d[lmax] <= 0.999:
            break
    a = np.mean(m, 2)[image_dark >= ht[1][lmax]].max()
     # 对值范围进行限制
    v1 = np.minimum(image_dark * w, max_v1)
    return v1, a


def haze(m, r=81, eps=0.001, w=0.95, max_v1=0.80, b_gamma=False):
    y = np.zeros(m.shape)
    # 得到遮罩图像和大气光照
    mask_img, a = fog(m, r, eps, w, max_v1)

    for k in range(3):
        # 颜色校正
        y[:,:,k] = (m[:,:,k] - mask_img) / (1 - mask_img / a)
    y = np.clip(y, 0, 1)
    if b_gamma:
        # gamma校正,默认不进行该操作
        y = y ** (np.log(0.5) / np.log(y.mean()))
    return y


In [78]:
if __name__ == '__main__':
    image_pos = haze(cv2.imread('images/1.jpg') / 255.0) * 255
    cv2.imwrite('worker1/result/1_pos.png', image_pos)